In [ ]:
import json

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

def get_traveler_name3(text):
    traveler_name = ''
    debarred = ['netlink', 'flat', 'date', 'transaction', 'gst', 'property', 'information', 'floor', 'tower']
    flag = 0
    for line in text:

        if 'room no' in line.lower() or 'room number' in line.lower():
            val = 'guest name'
            if not any(val in x.lower() for x in text):
                words = line.split()
                newwords = [word.lower() for word in words]
                ind = newwords.index('room')
                traveler_name = ' '.join(words[:ind])
                for deb in debarred:
                    if deb in traveler_name.lower():
                        traveler_name = ''
                        flag = 1
                        break
                if flag == 0:
                    return traveler_name
        flag = 0
        if 'id' in line.lower().split():
            words = line.split()
            newwords = [word.lower() for word in words]
            for newword in newwords:
                if newword in debarred:
                    flag = 1
                    break
            if flag == 0:
                ind = newwords.index('id')
                traveler_name = ' '.join(words[:ind])
                return traveler_name
        flag = 0
        if 'folio' in line.lower():
            words = line.split()
            newwords = [word.lower() for word in words]
            for newword in newwords:
                if newword in debarred:
                    flag = 1
                    break
            if flag == 0:
                ind = newwords.index('folio')
                traveler_name = ' '.join(words[:ind])
                return traveler_name
        flag = 0
        if 'guest name' in line.lower():
            words = line.split()
            newwords = [word.lower() for word in words]
            if 'departure' in newwords:
                ind = newwords.index('departure')
                traveler_name = ' '.join(words[3:ind])
                return traveler_name
            else:
                traveler_name = ' '.join(words[3:])
                return traveler_name
        flag = 0
        val = 'guest name'
        if not any(val in x.lower() for x in text):
            if 'mr' in line.lower() and 'omr' not in line.lower():
                words = line.split()
                newwords = [word.lower() for word in words]
                for newword in newwords:
                    if newword in debarred:
                        flag = 1
                        break
                if flag == 0:
                    if 'room' not in line.lower():
                        traveler_name = line
                        return traveler_name
                    else:
                        words = line.split()
                        newwords = [word.lower() for word in words]
                        ind = newwords.index('room')
                        traveler_name = ' '.join(words[:ind])
                        return traveler_name

def get_start_date3(text):
    start_date = ''
    formats = ['%d/%m/%Y', '%d-%b-%y', '%d-%m-%y']
    for line in text:
        if 'arrival' in line.lower():
            words = line.split()
            for word in words:
                if is_date(word.strip(':')):
                    start_date = word.strip(':')
                    for format in formats:
                        try:
                            start_date = datetime.datetime.strptime(start_date, format).strftime('%Y-%m-%d')
                        except:
                            pass
                    return start_date

def get_end_date3(text):
    end_date = ''
    formats = ['%d/%m/%Y', '%d-%b-%y', '%d-%m-%y']
    for line in text:
        if 'departure' in line.lower():
            words = line.split()
            newwords = line.lower().split()
            ind = newwords.index('departure')
            words = words[ind:]
            for word in words:
                if is_date(word.strip(':')):
                    end_date = word.strip(':')
                    for format in formats:
                        try:
                            end_date = datetime.datetime.strptime(end_date, format).strftime('%Y-%m-%d')
                        except:
                            pass
                    return end_date

def get_no_of_rooms3(text):
    rooms = 0
    for line in text:
        if 'room no' in line.lower() or 'room number' in line.lower():
            words = line.lower().split()
            print(words)
            ind = words.index('room')
            words = words[ind:]
            for word in words:
                if word.isnumeric():
                    rooms += 1
            return rooms

def get_no_of_nights3(text):
    no_of_nights = 0
    try:
        start_date = get_start_date3(text)
        fyear, fmonth, fday = start_date.split('-')
        fdate = datetime.date(int(fyear), int(fmonth), int(fday))
        end_date = get_end_date3(text)
        lyear, lmonth, lday = end_date.split('-')
        ldate = datetime.date(int(lyear), int(lmonth), int(lday))
        return (ldate-fdate).days
    except:
        pass

def get_hotel_name3(text):
    hotel_name = ''
    for line in text:
        if 'hilton garden inn' in line.lower():
            hotel_name = 'Hilton Garden Inn'
            break
        if 'doubletree by hilton' in line.lower():
            hotel_name = 'DOUBLETREE BY HILTON'
            break
        if 'park hyatt' in line.lower():
            hotel_name = 'PARK HYATT'
            break
        if 'courtyard by marriott' in line.lower():
            hotel_name = 'Courtyard by Marriott'
            break
        if 'park hotels' in line.lower():
            hotel_name = 'THE PARK Hotels'
            break
    return hotel_name

def replace(string, ch):
    string = string.replace(ch, '')
    return string

def get_hotel_city3(text):
    hotel_city = ''
    debarred = ['page']
    for line in text:
        words = line.split()
        for word in words:
            if word.lower() in cities:
                if word.lower() not in debarred:
                    hotel_city = word
                return hotel_city

def get_hotel_country3(text, data):
    hotel_country = ''
    hotel_city = get_hotel_city3(text)
    if hotel_city:
        hotel_city = hotel_city[0].upper()+hotel_city[1:].lower()
        value = data[data['city'] == hotel_city.strip()].iloc[0]
        hotel_country = value['country']
        return hotel_country

f = open('cities.json')
cities = json.load(f)
cities = {k.lower(): v for k, v in cities.items()}
f.close()
data = pd.read_excel('World_DB.xlsx')

for file in glob.glob('ensemble_files_hotel_stay4/*.txt'):
    print(file)

    with open(file) as fp:
        traveler_name_final = 0
        start_date_final = 0
        end_date_final = 0
        no_of_rooms_final = 0
        no_of_nights_final = 0
        hotel_name_final = 0
        hotel_city_final = 0
        hotel_country_final = 0

        texts = fp.read().split("---\n")
        # pprint(texts)
        for text in texts:
            text = text.splitlines()
            text = [line for line in text if not line.isspace() and len(line) > 0]
            filename = os.path.split(file)[-1].split('.txt')[0]

            traveler_name = get_traveler_name3(text)
            if traveler_name != '' and not traveler_name_final:
                traveler_name_final = traveler_name
            start_date = get_start_date3(text)
            if start_date != '' and not start_date_final:
                start_date_final = start_date
            end_date = get_end_date3(text)
            if end_date != '' and not end_date_final:
                end_date_final = end_date
            no_of_nights = get_no_of_nights3(text)
            if no_of_nights != '' and not no_of_nights_final:
                no_of_nights_final = no_of_nights
            hotel_name = get_hotel_name3(text)
            if hotel_name != '' and not hotel_name_final:
                hotel_name_final = hotel_name
            hotel_city = get_hotel_city3(text)
            if hotel_city != '' and not hotel_city_final:
                hotel_city_final = hotel_city
            hotel_country = get_hotel_country3(text, data)
            if hotel_country != '' and not hotel_country_final:
                hotel_country_final = hotel_country
            no_of_rooms = get_no_of_rooms3(text)
            if no_of_rooms != '' and not no_of_rooms_final:
                no_of_rooms_final = no_of_rooms

    print(traveler_name_final)
    print(start_date_final)
    print(end_date_final)
    print(no_of_rooms_final)
    print(no_of_nights_final)
    print(hotel_name_final)
    print(hotel_city_final)
    print(hotel_country_final)
    print('----------------------------------------')